In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [48]:
train = pd.read_csv("final_data_train.csv")
test = pd.read_csv("final_data_test.csv")
y = train['dropout']
data = pd.concat([ train.drop(['dropout'], axis=1), test.drop(['dropout'], axis=1)], axis=0,ignore_index=True, sort=False) 

In [211]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24013 entries, 0 to 24012
Data columns (total 29 columns):
enrollment_id                     24013 non-null int64
dropout                           24013 non-null int64
username                          24013 non-null object
course_id                         24013 non-null object
from                              24013 non-null object
to                                24013 non-null object
nCourses                          24013 non-null int64
nEnrollments                      24013 non-null int64
avg_delay_chapters                7514 non-null float64
avg_delay_chapters_course         24013 non-null float64
lifetime                          24013 non-null int64
btw_course_gap_min                24013 non-null int64
btw_course_gap_max                24013 non-null int64
btw_course_gap_mean               24013 non-null float64
access                            24013 non-null int64
discussion                        24013 non-null int64
na

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96408 entries, 0 to 96407
Data columns (total 28 columns):
enrollment_id                     96408 non-null int64
username                          96408 non-null object
course_id                         96408 non-null object
from                              96408 non-null object
to                                96408 non-null object
nCourses                          96408 non-null int64
nEnrollments                      96408 non-null int64
avg_delay_chapters                30172 non-null float64
avg_delay_chapters_course         96408 non-null float64
lifetime                          96408 non-null int64
btw_course_gap_min                96408 non-null int64
btw_course_gap_max                96408 non-null int64
btw_course_gap_mean               96408 non-null float64
access                            96408 non-null int64
discussion                        96408 non-null int64
navigate                          96408 non-null int64
p

In [114]:
labels = [
 'nCourses',
 'nEnrollments',
 'lifetime_group',
 'access_group',
 'btw_course_gap_mean',
 'discussion',
 'navigate',
 'page_close',
 'problem',
 'video',
 'wiki',
 'problem_duration',
 'video_duration',
 'problem_duration_per_course',
 'video_duration_per_course']

In [50]:
data['access_group'] = pd.cut(data['access'], [-1,100,200,300,400,500,2200])

In [51]:
data['lifetime_group'] = pd.cut(data['lifetime'], [-1, 52, 110, 190,280 ])

In [52]:
data['md_interarrival_time'] = data['md_interarrival_time'].astype('int')

In [53]:
data['mean_interarrival_time'] = data['mean_interarrival_time'].astype('int')

In [54]:
data['problem_duration_per_course'] = data['problem_duration_per_course'].astype('int')

In [55]:
data['video_duration_per_course'] = data['video_duration_per_course'].astype('int')

In [56]:
data['discussion_duration_per_course'] = data['discussion_duration_per_course'].astype('int')

In [63]:
data['btw_course_gap_mean'] = data['btw_course_gap_mean'].astype('int')

In [ ]:
T = data[labels]

from sklearn.preprocessing import StandardScaler as SSC
ssc = SSC()
d = T[[]]
d =  pd.DataFrame(ssc.fit_transform(d), columns=['farestd'])

In [115]:

D = pd.get_dummies(T)
TRAIN = D[:len(train)]
TEST = D[len(train):]

In [116]:
T.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96408 entries, 0 to 96407
Data columns (total 15 columns):
nCourses                       96408 non-null int64
nEnrollments                   96408 non-null int64
lifetime_group                 96408 non-null category
access_group                   96407 non-null category
btw_course_gap_mean            96408 non-null int64
discussion                     96408 non-null int64
navigate                       96408 non-null int64
page_close                     96408 non-null int64
problem                        96408 non-null int64
video                          96408 non-null int64
wiki                           96408 non-null int64
problem_duration               96408 non-null int64
video_duration                 96408 non-null int64
problem_duration_per_course    96408 non-null int64
video_duration_per_course      96408 non-null int64
dtypes: category(2), int64(13)
memory usage: 9.7 MB


In [117]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(TRAIN, y, test_size=0.2, random_state=0)

In [130]:
from sklearn.model_selection import cross_val_score as CVS

CatClassifier
================================================

In [118]:
cat_features = list(range(0, TRAIN.shape[1]))

In [214]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=650,
    learning_rate=0.25,
    random_seed=0
    #loss_function='CrossEntropy'
)



In [195]:
clf.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_val, y_val), 
        verbose=False
)

In [121]:
clf.feature_importances_

In [196]:
clf.best_score_

{'learn': {'Logloss': 0.27891501760689646},
 'validation': {'Logloss': 0.3385270195898288}}

In [200]:
cv = CVS(clf, TRAIN, y,cv=10, scoring='accuracy')


0:	learn: 0.5238338	total: 31.6ms	remaining: 20.5s
1:	learn: 0.4445836	total: 64.6ms	remaining: 20.9s
2:	learn: 0.4021284	total: 97.5ms	remaining: 21s
3:	learn: 0.3806196	total: 131ms	remaining: 21.1s
4:	learn: 0.3677653	total: 164ms	remaining: 21.1s
5:	learn: 0.3599226	total: 195ms	remaining: 20.9s
6:	learn: 0.3548320	total: 227ms	remaining: 20.8s
7:	learn: 0.3508847	total: 287ms	remaining: 23.1s
8:	learn: 0.3489689	total: 322ms	remaining: 22.9s
9:	learn: 0.3472762	total: 339ms	remaining: 21.7s
10:	learn: 0.3458707	total: 353ms	remaining: 20.5s
11:	learn: 0.3452045	total: 363ms	remaining: 19.3s
12:	learn: 0.3442524	total: 375ms	remaining: 18.4s
13:	learn: 0.3432788	total: 386ms	remaining: 17.5s
14:	learn: 0.3427321	total: 397ms	remaining: 16.8s
15:	learn: 0.3419777	total: 408ms	remaining: 16.2s
16:	learn: 0.3416678	total: 419ms	remaining: 15.6s
17:	learn: 0.3408370	total: 432ms	remaining: 15.2s
18:	learn: 0.3402893	total: 445ms	remaining: 14.8s
19:	learn: 0.3398117	total: 456ms	remain

173:	learn: 0.3044502	total: 2.34s	remaining: 6.4s
174:	learn: 0.3043011	total: 2.36s	remaining: 6.4s
175:	learn: 0.3041934	total: 2.37s	remaining: 6.39s
176:	learn: 0.3040091	total: 2.39s	remaining: 6.38s
177:	learn: 0.3038517	total: 2.4s	remaining: 6.37s
178:	learn: 0.3037403	total: 2.42s	remaining: 6.36s
179:	learn: 0.3035817	total: 2.43s	remaining: 6.34s
180:	learn: 0.3033276	total: 2.44s	remaining: 6.33s
181:	learn: 0.3030857	total: 2.46s	remaining: 6.32s
182:	learn: 0.3027988	total: 2.47s	remaining: 6.3s
183:	learn: 0.3026665	total: 2.48s	remaining: 6.29s
184:	learn: 0.3025138	total: 2.5s	remaining: 6.27s
185:	learn: 0.3024068	total: 2.51s	remaining: 6.26s
186:	learn: 0.3021721	total: 2.52s	remaining: 6.25s
187:	learn: 0.3020941	total: 2.54s	remaining: 6.23s
188:	learn: 0.3020079	total: 2.56s	remaining: 6.24s
189:	learn: 0.3017940	total: 2.57s	remaining: 6.23s
190:	learn: 0.3015428	total: 2.59s	remaining: 6.22s
191:	learn: 0.3013451	total: 2.6s	remaining: 6.21s
192:	learn: 0.3011

342:	learn: 0.2808899	total: 4.42s	remaining: 3.96s
343:	learn: 0.2808310	total: 4.44s	remaining: 3.95s
344:	learn: 0.2805989	total: 4.45s	remaining: 3.94s
345:	learn: 0.2805045	total: 4.47s	remaining: 3.92s
346:	learn: 0.2804166	total: 4.48s	remaining: 3.91s
347:	learn: 0.2803020	total: 4.49s	remaining: 3.9s
348:	learn: 0.2802739	total: 4.5s	remaining: 3.88s
349:	learn: 0.2801598	total: 4.51s	remaining: 3.87s
350:	learn: 0.2800018	total: 4.52s	remaining: 3.85s
351:	learn: 0.2799097	total: 4.53s	remaining: 3.84s
352:	learn: 0.2797671	total: 4.54s	remaining: 3.82s
353:	learn: 0.2797113	total: 4.55s	remaining: 3.81s
354:	learn: 0.2795923	total: 4.57s	remaining: 3.79s
355:	learn: 0.2793992	total: 4.58s	remaining: 3.78s
356:	learn: 0.2792304	total: 4.59s	remaining: 3.77s
357:	learn: 0.2791243	total: 4.6s	remaining: 3.75s
358:	learn: 0.2790475	total: 4.61s	remaining: 3.74s
359:	learn: 0.2789401	total: 4.63s	remaining: 3.73s
360:	learn: 0.2787862	total: 4.64s	remaining: 3.72s
361:	learn: 0.2

516:	learn: 0.2621113	total: 6.51s	remaining: 1.68s
517:	learn: 0.2620314	total: 6.53s	remaining: 1.67s
518:	learn: 0.2618965	total: 6.55s	remaining: 1.65s
519:	learn: 0.2618086	total: 6.56s	remaining: 1.64s
520:	learn: 0.2617069	total: 6.57s	remaining: 1.63s
521:	learn: 0.2616061	total: 6.58s	remaining: 1.61s
522:	learn: 0.2614726	total: 6.59s	remaining: 1.6s
523:	learn: 0.2613739	total: 6.6s	remaining: 1.59s
524:	learn: 0.2613159	total: 6.61s	remaining: 1.57s
525:	learn: 0.2612688	total: 6.63s	remaining: 1.56s
526:	learn: 0.2611768	total: 6.64s	remaining: 1.55s
527:	learn: 0.2611167	total: 6.65s	remaining: 1.54s
528:	learn: 0.2610794	total: 6.66s	remaining: 1.52s
529:	learn: 0.2609371	total: 6.67s	remaining: 1.51s
530:	learn: 0.2608642	total: 6.68s	remaining: 1.5s
531:	learn: 0.2607961	total: 6.69s	remaining: 1.48s
532:	learn: 0.2606746	total: 6.71s	remaining: 1.47s
533:	learn: 0.2606231	total: 6.72s	remaining: 1.46s
534:	learn: 0.2605211	total: 6.74s	remaining: 1.45s
535:	learn: 0.2

25:	learn: 0.3443238	total: 468ms	remaining: 11.2s
26:	learn: 0.3440433	total: 483ms	remaining: 11.2s
27:	learn: 0.3437344	total: 497ms	remaining: 11s
28:	learn: 0.3434626	total: 508ms	remaining: 10.9s
29:	learn: 0.3430709	total: 519ms	remaining: 10.7s
30:	learn: 0.3428372	total: 530ms	remaining: 10.6s
31:	learn: 0.3424824	total: 541ms	remaining: 10.5s
32:	learn: 0.3422104	total: 552ms	remaining: 10.3s
33:	learn: 0.3420573	total: 563ms	remaining: 10.2s
34:	learn: 0.3417841	total: 574ms	remaining: 10.1s
35:	learn: 0.3414815	total: 585ms	remaining: 9.97s
36:	learn: 0.3412521	total: 596ms	remaining: 9.87s
37:	learn: 0.3409846	total: 607ms	remaining: 9.78s
38:	learn: 0.3408059	total: 617ms	remaining: 9.67s
39:	learn: 0.3403629	total: 628ms	remaining: 9.58s
40:	learn: 0.3400368	total: 638ms	remaining: 9.48s
41:	learn: 0.3397600	total: 649ms	remaining: 9.4s
42:	learn: 0.3393855	total: 661ms	remaining: 9.33s
43:	learn: 0.3389327	total: 678ms	remaining: 9.34s
44:	learn: 0.3385046	total: 699ms	

...

633:	learn: 0.2543011	total: 9.93s	remaining: 251ms
634:	learn: 0.2541286	total: 9.94s	remaining: 235ms
635:	learn: 0.2540369	total: 9.96s	remaining: 219ms
636:	learn: 0.2540247	total: 9.97s	remaining: 203ms
637:	learn: 0.2539307	total: 9.98s	remaining: 188ms
638:	learn: 0.2538424	total: 9.99s	remaining: 172ms
639:	learn: 0.2537728	total: 10s	remaining: 156ms
640:	learn: 0.2536274	total: 10s	remaining: 141ms
641:	learn: 0.2535325	total: 10s	remaining: 125ms
642:	learn: 0.2534218	total: 10s	remaining: 109ms
643:	learn: 0.2533989	total: 10.1s	remaining: 93.7ms
644:	learn: 0.2533713	total: 10.1s	remaining: 78.1ms
645:	learn: 0.2533287	total: 10.1s	remaining: 62.4ms
646:	learn: 0.2532596	total: 10.1s	remaining: 46.8ms
647:	learn: 0.2532092	total: 10.1s	remaining: 31.2ms
648:	learn: 0.2530810	total: 10.1s	remaining: 15.6ms
649:	learn: 0.2529826	total: 10.1s	remaining: 0us
0:	learn: 0.5145586	total: 30ms	remaining: 19.5s
1:	learn: 0.4309578	total: 61.2ms	remaining: 19.8s
2:	learn: 0.3881878	

145:	learn: 0.2958980	total: 2.57s	remaining: 8.88s
146:	learn: 0.2957813	total: 2.59s	remaining: 8.87s
147:	learn: 0.2955942	total: 2.61s	remaining: 8.85s
148:	learn: 0.2953176	total: 2.63s	remaining: 8.83s
149:	learn: 0.2951415	total: 2.64s	remaining: 8.8s
150:	learn: 0.2949875	total: 2.65s	remaining: 8.76s
151:	learn: 0.2947811	total: 2.66s	remaining: 8.72s
152:	learn: 0.2945361	total: 2.67s	remaining: 8.69s
153:	learn: 0.2942923	total: 2.69s	remaining: 8.65s
154:	learn: 0.2942002	total: 2.7s	remaining: 8.62s
155:	learn: 0.2939176	total: 2.72s	remaining: 8.61s
156:	learn: 0.2937270	total: 2.73s	remaining: 8.57s
157:	learn: 0.2935124	total: 2.74s	remaining: 8.54s
158:	learn: 0.2933072	total: 2.76s	remaining: 8.53s
159:	learn: 0.2930846	total: 2.78s	remaining: 8.51s
160:	learn: 0.2928795	total: 2.8s	remaining: 8.52s
161:	learn: 0.2927102	total: 2.82s	remaining: 8.5s
162:	learn: 0.2924864	total: 2.83s	remaining: 8.46s
163:	learn: 0.2922734	total: 2.84s	remaining: 8.43s
164:	learn: 0.29

307:	learn: 0.2716898	total: 5.29s	remaining: 5.87s
308:	learn: 0.2715386	total: 5.31s	remaining: 5.86s
309:	learn: 0.2714436	total: 5.33s	remaining: 5.85s
310:	learn: 0.2713506	total: 5.35s	remaining: 5.83s
311:	learn: 0.2712315	total: 5.36s	remaining: 5.8s
312:	learn: 0.2711238	total: 5.37s	remaining: 5.78s
313:	learn: 0.2710039	total: 5.38s	remaining: 5.76s
314:	learn: 0.2708964	total: 5.39s	remaining: 5.74s
315:	learn: 0.2707523	total: 5.4s	remaining: 5.71s
316:	learn: 0.2706591	total: 5.42s	remaining: 5.69s
317:	learn: 0.2705331	total: 5.43s	remaining: 5.67s
318:	learn: 0.2704886	total: 5.45s	remaining: 5.65s
319:	learn: 0.2703154	total: 5.48s	remaining: 5.65s
320:	learn: 0.2701501	total: 5.51s	remaining: 5.64s
321:	learn: 0.2700561	total: 5.54s	remaining: 5.65s
322:	learn: 0.2699120	total: 5.58s	remaining: 5.65s
323:	learn: 0.2698233	total: 5.61s	remaining: 5.65s
324:	learn: 0.2697600	total: 5.64s	remaining: 5.64s
325:	learn: 0.2696333	total: 5.67s	remaining: 5.64s
326:	learn: 0.

466:	learn: 0.2530660	total: 8.23s	remaining: 3.22s
467:	learn: 0.2529619	total: 8.24s	remaining: 3.21s
468:	learn: 0.2528693	total: 8.26s	remaining: 3.19s
469:	learn: 0.2527977	total: 8.28s	remaining: 3.17s
470:	learn: 0.2526621	total: 8.29s	remaining: 3.15s
471:	learn: 0.2525289	total: 8.3s	remaining: 3.13s
472:	learn: 0.2524021	total: 8.31s	remaining: 3.11s
473:	learn: 0.2523420	total: 8.33s	remaining: 3.09s
474:	learn: 0.2522567	total: 8.34s	remaining: 3.07s
475:	learn: 0.2521570	total: 8.35s	remaining: 3.05s
476:	learn: 0.2520436	total: 8.36s	remaining: 3.03s
477:	learn: 0.2519501	total: 8.38s	remaining: 3.01s
478:	learn: 0.2518301	total: 8.39s	remaining: 3s
479:	learn: 0.2517279	total: 8.4s	remaining: 2.98s
480:	learn: 0.2516415	total: 8.41s	remaining: 2.96s
481:	learn: 0.2515634	total: 8.43s	remaining: 2.94s
482:	learn: 0.2514774	total: 8.44s	remaining: 2.92s
483:	learn: 0.2513786	total: 8.46s	remaining: 2.9s
484:	learn: 0.2512928	total: 8.47s	remaining: 2.88s
485:	learn: 0.2512

626:	learn: 0.2375835	total: 10.5s	remaining: 387ms
627:	learn: 0.2375296	total: 10.6s	remaining: 370ms
628:	learn: 0.2374753	total: 10.6s	remaining: 353ms
629:	learn: 0.2373383	total: 10.6s	remaining: 336ms
630:	learn: 0.2372630	total: 10.6s	remaining: 319ms
631:	learn: 0.2371743	total: 10.6s	remaining: 303ms
632:	learn: 0.2371169	total: 10.6s	remaining: 286ms
633:	learn: 0.2370100	total: 10.7s	remaining: 269ms
634:	learn: 0.2369262	total: 10.7s	remaining: 253ms
635:	learn: 0.2368512	total: 10.7s	remaining: 236ms
636:	learn: 0.2367862	total: 10.7s	remaining: 219ms
637:	learn: 0.2366831	total: 10.7s	remaining: 202ms
638:	learn: 0.2365329	total: 10.8s	remaining: 185ms
639:	learn: 0.2364262	total: 10.8s	remaining: 168ms
640:	learn: 0.2363839	total: 10.8s	remaining: 152ms
641:	learn: 0.2363414	total: 10.8s	remaining: 135ms
642:	learn: 0.2362471	total: 10.8s	remaining: 118ms
643:	learn: 0.2361838	total: 10.9s	remaining: 101ms
644:	learn: 0.2361323	total: 10.9s	remaining: 84.2ms
645:	learn:

In [201]:
cv.mean()

0.855707401398854

In [205]:
clf.fit(TRAIN, y, cat_features=cat_features, verbose=False)

In [206]:
yp = clf.predict(TEST)

In [212]:
(yp == test['dropout']).sum()

20271

In [213]:
20271/24013

0.8441677424728272